In [450]:
from collections import defaultdict
from copy import deepcopy
from itertools import chain, combinations
import numpy as np

In [704]:

input_file = "example_input"
input_file = "input"

input_text = open(input_file).read()

def parse_input(inp):
    scanners = []
    scanner = []
    for line in inp.splitlines():
        if not line.strip():
            scanners.append(scanner)
            scanner = []
        elif '---' not in line:
            scanner.append(np.array(list(map(int, line.split(',')))))
    scanners.append(scanner)
    return scanners



In [460]:
def get_beacon_distances(scanner):
    distances = defaultdict(list)
    for i in range(len(scanner)):
        src = scanner[i]
        for j in range(i + 1, len(scanner)):
            dist = np.linalg.norm(src - scanner[j])
            if dist in distances:
                raise ValueError("Dupe distance", i)
            distances[dist] = (i, j)
    return distances


In [705]:
scanners = parse_input(input_text)
beacon_distances_to_ids = [get_beacon_distances(sc) for sc in scanners]

In [706]:
id_maps = {}
for i in range(len(beacon_distances_to_ids)):
    src_ds = beacon_distances_to_ids[i]
    src_id_maps = {}
    for j in range(i + 1, len(beacon_distances_to_ids)):
        dst_ds = beacon_distances_to_ids[j]

        distances = set(src_ds).intersection(dst_ds)

        src_ids = set(chain(*[src_ds[k] for k in distances]))
        dst_ids = set(chain(*[dst_ds[k] for k in distances]))

        id_map = {}
        for dist1, dist2 in combinations(distances, 2):
            common = set(src_ds[dist1]).intersection(src_ds[dist2])
            if common and not common.intersection(id_map):
                common2 = set(dst_ds[dist1]).intersection(dst_ds[dist2])

                id_map[list(common)[0]] = list(common2)[0]

        src_id_maps[j] = id_map
    id_maps[i] = src_id_maps


In [707]:
id_maps_full = deepcopy(id_maps)
for sca, v in id_maps.items():
    for scb, mapping in v.items():
        id_maps_full[scb][sca] = dict(ids[::-1] for ids in mapping.items())

id_maps_12 = {}
for k, v in id_maps.items():
    id_maps_12[k] = {k2: v2 for k2, v2 in v.items() if len(v2) >= 12}
    
id_maps_12_full = deepcopy(id_maps_12)
for sca, v in id_maps_12.items():
    for scb, mapping in v.items():
        id_maps_12_full[scb][sca] = dict(ids[::-1] for ids in mapping.items())

In [687]:
for a, v in id_maps.items():
    for b, v2 in v.items():
        print(a, b, len(v2))

0 1 12
0 2 3
0 3 0
0 4 6
1 2 6
1 3 12
1 4 12
2 3 3
2 4 12
3 4 6


In [ ]:
def get_beacon_distances(scanner):
    distances = defaultdict(list)
    for i in range(len(scanner)):
        src = scanner[i]
        for j in range(i + 1, len(scanner)):
            dist = np.linalg.norm(src - scanner[j])
            if dist in distances:
                raise ValueError("Dupe distance", i)
            distances[dist] = (i, j)
    return distances


In [710]:
# Get offset. There is assuredly a fancy way of doing this. Do it the dumb way
offsets = defaultdict(dict)
for sc1, v in id_maps_12_full.items():
    for sc2, v2 in v.items():
        # if sc1 != 0 or sc2 != 1:
        #     continue
        
        if len(v2) < 3:
            continue
        
        idx_pairs = list(v2.items())
        for i in range(len(idx_pairs) - 1):
            sc1_id1, sc2_id1 = idx_pairs[i]
            sc1_id2, sc2_id2 = idx_pairs[i + 1]

            sc1_id1_pos = scanners[sc1][sc1_id1]
            sc1_id2_pos = scanners[sc1][sc1_id2]
            sc2_id1_pos = scanners[sc2][sc2_id1]
            sc2_id2_pos = scanners[sc2][sc2_id2]
            
            sc1_offset = sc1_id2_pos - sc1_id1_pos
            sc2_offset = sc2_id2_pos - sc2_id1_pos

            sc1_abs = abs(sc1_offset)
            sc2_abs = abs(sc2_offset)

            if len(np.unique(sc1_abs)) != 3 or len(np.unique(sc2_abs)) != 3:
                continue

            sc2_idx_mapping = [np.where(sc2_abs == v)[0][0] for v in sc1_abs]
            sc2_offset_remapped = [sc2_offset[i] for i in sc2_idx_mapping]
            sc2_sign_mapping = sc1_offset / sc2_offset_remapped

            sc1_idx_mapping = [np.where(sc1_abs == v)[0][0] for v in sc2_abs]
            sc1_offset_remapped = [sc1_offset[i] for i in sc1_idx_mapping]
            sc1_sign_mapping = sc2_offset / sc1_offset_remapped

            sc2_conv = [sc2_id1_pos[i] for i in sc2_idx_mapping]
            sc2_conv_full = sc2_conv * sc2_sign_mapping
            sc2_pos_offset = sc1_id1_pos - sc2_conv_full
            sc1_pos_offset = sc2_conv_full - sc1_id1_pos

            sc2_offsets = {'idx': sc2_idx_mapping, 'sign': sc2_sign_mapping, 'pos': sc2_pos_offset}
            offsets[sc2][sc1] = sc2_offsets

            sc1_offsets = {'idx': sc1_idx_mapping, 'sign': sc1_sign_mapping, 'pos': sc1_pos_offset}
            # offsets[sc1][sc2] = sc1_offsets

            print(sc2, sc1, sc2_idx_mapping, sc2_sign_mapping, sc2_pos_offset)
            # print(sc1, sc2, sc1_idx_mapping, sc1_sign_mapping, sc1_pos_offset)
            break

1 0 [0, 2, 1] [ 1. -1.  1.] [   64.  -168. -1295.]
25 1 [1, 2, 0] [-1.  1. -1.] [  -55. -1099.  -121.]
0 1 [0, 2, 1] [ 1.  1. -1.] [ -64. 1295. -168.]
13 2 [1, 2, 0] [-1. -1.  1.] [-151. 1301.  156.]
27 2 [0, 2, 1] [ 1. -1.  1.] [   21. -1128.    36.]
16 3 [0, 1, 2] [-1. -1.  1.] [ -67.  -34. 1269.]
19 3 [1, 0, 2] [ 1. -1.  1.] [1140.   53.  -57.]
20 3 [1, 0, 2] [-1.  1.  1.] [   3. 1256.   34.]
22 3 [2, 0, 1] [-1.  1. -1.] [ -157.    65. -1193.]
26 3 [2, 1, 0] [-1.  1.  1.] [ -101. -1136.   -39.]
27 4 [2, 0, 1] [-1. -1.  1.] [  12. 1185. -122.]
30 4 [1, 2, 0] [-1. -1.  1.] [-1145.   163.  -167.]
37 4 [2, 1, 0] [-1.  1.  1.] [ -116.   107. -1262.]
8 5 [2, 0, 1] [-1.  1. -1.] [1155. -132.   62.]
11 6 [2, 1, 0] [-1.  1.  1.] [  -44. -1328.   123.]
33 6 [1, 0, 2] [-1.  1.  1.] [1120. -121.  -40.]
8 7 [0, 1, 2] [ 1. -1. -1.] [  -15. -1141.    70.]
14 7 [0, 1, 2] [-1. -1.  1.] [-1136.   106.   -38.]
29 7 [1, 0, 2] [ 1.  1. -1.] [  91.  147. 1158.]
16 8 [2, 0, 1] [ 1. -1. -1.] [-1102.    25.

In [691]:
offsets

defaultdict(dict,
            {1: {0: {'idx': [0, 1, 2],
               'sign': array([-1.,  1., -1.]),
               'pos': array([   68., -1246.,   -43.])},
              2: {'idx': [0, 2, 1],
               'sign': array([ 1., -1.,  1.]),
               'pos': array([ 1037., -1272.,   -41.])},
              3: {'idx': [0, 1, 2],
               'sign': array([1., 1., 1.]),
               'pos': array([-160., 1134.,   23.])},
              4: {'idx': [2, 0, 1],
               'sign': array([-1.,  1., -1.]),
               'pos': array([-1104.,   -88.,   113.])}},
             2: {0: {'idx': [0, 2, 1],
               'sign': array([-1.,  1.,  1.]),
               'pos': array([ 1105., -1205.,  1229.])},
              1: {'idx': [0, 2, 1],
               'sign': array([ 1.,  1., -1.]),
               'pos': array([-1037.,    41., -1272.])},
              3: {'idx': [0, 2, 1],
               'sign': array([ 1.,  1., -1.]),
               'pos': array([-1197.,  1175., -1249.])},
        

In [582]:
(sc1_id1_pos - sc2_conv_full) - np.array([   68., -1246.,   -43.])

array([   20.,  1359., -1061.])

In [575]:
sc1_id1_pos - sc2_conv_full

array([   68., -1246.,   -43.])

In [ ]:
def get_beacon_distances(scanner):
    distances = defaultdict(list)
    for i in range(len(scanner)):
        src = scanner[i]
        for j in range(i + 1, len(scanner)):
            dist = np.linalg.norm(src - scanner[j])
            if dist in distances:
                raise ValueError("Dupe distance", i)
            distances[dist] = (i, j)
    return distances


In [698]:
def get_route_to_zero(offsets, path):
    idx = path[-1]
    if idx == 0:
        return path
    if 0 in offsets[idx]:
        return path + [0]
    paths = []
    for next_idx in offsets[idx]:
        if next_idx not in path:
            p = get_route_to_zero(offsets, path + [next_idx])
            if p:
                paths.append(p)
    return sorted(paths)[0] if paths else None

In [455]:
# offsets_copy = deepcopy(offsets)
# offsets_to_zero = {}
# for i in range(1, len(scanners)):
#     print(f'For {i} to 0')
#     route_to_zero = get_route_to_zero(offsets_copy, [i])[1:]
#     if not route_to_zero:
#         continue
#     # print(route_to_zero)
#     next_id = route_to_zero.pop(0)
#     current_offset = offsets_copy[i][next_id]
#     curr_id = next_id
#     for next_id in route_to_zero:
#         # print(curr_id, '->', next_id)
#         next_offset = offsets_copy[curr_id][next_id]
#         # print(current_offset)
#         current_offset['sign'] *= next_offset['sign']
#         current_offset['pos'] += next_offset['pos']
#         current_offset['idx'] = [current_offset['idx'][i] for i in next_offset['idx']]
#         # print(current_offset)
#         curr_id = next_id
#     offsets_to_zero[i] = current_offset

In [612]:
def transform_point(point, offset):
    adjusted = [point[i] for i in offset['idx']]
    adjusted *= offset['sign']
    adjusted += offset['pos']
    return adjusted

In [314]:
def transform_point_to_scanner_zero(src_scanner_id, point):
    rtz = get_route_to_zero(offsets, [src_scanner_id])
    for curr_id, next_id in zip(rtz[:-1], rtz[1:]):
        point = transform_point_between_scanners(curr_id, next_id, point)
    return point

def transform_point_between_scanners(src_scanner_id, dst_scanner_id, point):
    # offset = offsets_to_zero[src_scanner]
    offset = offsets[src_scanner_id][dst_scanner_id]
    return transform_point(point, offset)

In [470]:
get_route_to_zero(offsets, [7])

[7, 8, 16, 3, 19, 25, 1, 0]

In [699]:
for i in range(len(scanners)):
    print(get_route_to_zero(offsets, [i]))

[0]
[1, 0]
[2, 0]
[3, 1, 0]
[4, 0]


In [703]:
src_scanner = 4
dst_scanner = 4
for src_id, dst_id in id_maps_12_full[src_scanner][dst_scanner].items():
    src_point = scanners[src_scanner][src_id]
    dst_point = scanners[dst_scanner][dst_id]
    # print('\t', pta)
    # adjusted = transform_point_to_scanner_zero(src_scanner, scanners[src_scanner][src_id])
    adjusted = transform_point_between_scanners(src_scanner, dst_scanner, src_point)
    # adjusted_zero = transform_point_between_scanners(dst_scanner, 0, adjusted)
    print(src_point, '->', dst_point, ':', adjusted, (adjusted == dst_point).all())

KeyError: 0

In [602]:
offsets[3][1], offsets[1][0]

({'idx': [0, 1, 2],
  'sign': array([1., 1., 1.]),
  'pos': array([-160., 1134.,   23.])},
 {'idx': [0, 1, 2],
  'sign': array([-1.,  1., -1.]),
  'pos': array([ -68., 1246.,   43.])})

In [565]:
(offsets[4][1]['pos'] 
# * offsets[4][1]['sign'] 
+ offsets[1][0]['pos'] 
* offsets[1][0]['sign']
)

array([ -20., 1133., 1061.])

In [711]:
all_points = scanners[0].copy()
for scanner_id in range(1, len(scanners)):
    for point in scanners[scanner_id]:
        all_points.append(transform_point_to_scanner_zero(scanner_id, point))


In [712]:
len(set(tuple(p) for p in all_points))

512

In [636]:
def get_scanner_location_from_zero(src_scanner_id):
    rtz = get_route_to_zero(offsets, [src_scanner_id])
    point = offsets[rtz[0]][rtz[1]]['pos']
    return transform_point_to_scanner_zero(rtz[1], point)

In [713]:
scanner_locations = []
for i in range(1, len(scanners)):
    scanner_locations.append(get_scanner_location_from_zero(i))

In [714]:
def manhattan_distance(a, b):
    return sum(abs(a_val - b_val) for a_val, b_val in zip(a, b))


In [715]:
m_dists = []
for a, b in list(combinations(scanner_locations, 2)):
    m_dists.append(manhattan_distance(a, b))

In [716]:
max(m_dists)

15739.0

In [717]:
yes = []
no = []
for src_scanner, dst_scanners in id_maps_12_full.items():
    for dst_scanner, id_map in dst_scanners.items():
        matched = []
        for src_id, dst_id in id_map.items():
            src_point = scanners[src_scanner][src_id]
            dst_point = scanners[dst_scanner][dst_id]
            adjusted = transform_point_between_scanners(src_scanner, dst_scanner, src_point)
            matched.append((adjusted == dst_point).all())
            # print(src_point, '->', dst_point, ':', adjusted, (adjusted == dst_point).all())
        if all(matched):
            yes.append((src_scanner, dst_scanner))
        else:
            no.append((src_scanner, dst_scanner))
        # print(src_scanner, '->', dst_scanner, all(matched))

print(len(yes))
print(len(no))

86
0


In [466]:
no

[(7, 29), (29, 7)]

In [324]:
t = '''-892,524,684
-876,649,763
-838,591,734
-789,900,-551
-739,-1745,668
-706,-3180,-659
-697,-3072,-689
-689,845,-530
-687,-1600,576
-661,-816,-575
-654,-3158,-753
-635,-1737,486
-631,-672,1502
-624,-1620,1868
-620,-3212,371
-618,-824,-621
-612,-1695,1788
-601,-1648,-643
-584,868,-557
-537,-823,-458
-532,-1715,1894
-518,-1681,-600
-499,-1607,-770
-485,-357,347
-470,-3283,303
-456,-621,1527
-447,-329,318
-430,-3130,366
-413,-627,1469
-345,-311,381
-36,-1284,1171
-27,-1108,-65
7,-33,-71
12,-2351,-103
26,-1119,1091
346,-2985,342
366,-3059,397
377,-2827,367
390,-675,-793
396,-1931,-563
404,-588,-901
408,-1815,803
423,-701,434
432,-2009,850
443,580,662
455,729,728
456,-540,1869
459,-707,401
465,-695,1988
474,580,667
496,-1584,1900
497,-1838,-617
527,-524,1933
528,-643,409
534,-1912,768
544,-627,-890
553,345,-567
564,392,-477
568,-2007,-577
605,-1665,1952
612,-1593,1893
630,319,-379
686,-3108,-505
776,-3184,-501
846,-3110,-434
1135,-1161,1235
1243,-1093,1063
1660,-552,429
1693,-557,386
1735,-437,1738
1749,-1800,1813
1772,-405,1572
1776,-675,371
1779,-442,1789
1780,-1548,337
1786,-1538,337
1847,-1591,415
1889,-1729,1762
1994,-1805,1792'''
beacons = [tuple(map(int, line.split(','))) for line in t.splitlines()]

In [360]:
sc0 = set(tuple(p) for p in scanners[0])

In [361]:
len(sc0)

25